## **Mapeamento Esquema Qatar**

##### **___Informações___**

**Projeto:** Qatar

**Autor:** Lucas Pimenta dos Santos Monteiro

**Data:** 17/12/2024

##### **___Objetivo___**

A ideia desse cógido é automatizar a visualização de uma imagem .svg gerada no Inkscape de acordo com o output do programa (report.csv) e os dados de entrada definidos (entry_data.csv).

Para isso foram criados rótulos no Inkscape que identificam cada elemento de texto a partir do inkscape:label. Cada label recebeu o nome equivalente a variável proveniente do report.csv e entry_data.csv que representa o valor desejado.

##### **Importando as Bibliotecas**

In [3]:
import pandas as pd

##### **Lendo o arquivo entry_data.csv e report.csv e armazenando-os em dataframes**

In [20]:
entry_data = pd.read_csv("entry_data.csv")
report = pd.read_csv("report.csv")

##### **Verificando os Dados**

In [21]:
entry_data.head()

,Entry data:,Unnamed: 1,Unnamed: 2
0,Vacuum pressure =,-500.0000000000,mbar
1,Membrane area =,12.9600000000,m²
2,Membrane thickness =,100.0000000000,microns
3,Membrane porosity =,85.0000000000%,NaN
4,Pore diameter =,0.3200000000,microns


In [22]:
report.head()

,Time (s),Desal - Coolant mass flow rate (kg/h),Desal - Hot feedwater inlet temperature (°C),Desal - Hot feedwater outlet temperature (°C),Desal - Cold feedwater inlet temperature (°C),Desal - Cold feedwater outlet temperature (°C),Desal - Feed membrane temperature (°C),Desal - Membrane gap temperature (°C),Desal - Gap film boundary temperature (°C),Desal - Film wall temperature (°C),...,Turbine - Regenerator cold side outlet temperature (°C),Turbine - Combusiton chamber outlet temperature (°C),Turbine - Turbine outlet temperature (ISENTROPIC) (°C),Turbine - Turbine outlet temperature (REAL) (°C),Turbine - Regenerator hot side outlet temperature (°C),Turbine - Compressor power (W),Turbine - Turbine power (W),Turbine - Net power (W),Turbine - Cycle efficiency,Unnamed: 79
0,126.0,1059.647208,0.391759,43.266656,29.936430,66.562169,60.687908,56.982331,53.000174,49.945944,...,391.001621,703.641577,311.695918,404.38948,284.198075,-32841.046769,43509.148381,10668.101612,0.23498,NaN
1,252.0,1060.125744,0.387233,43.456500,30.136843,66.571523,60.783462,57.068474,53.076148,50.043236,...,391.001621,703.641577,311.695918,404.38948,284.198075,-32841.046769,43509.148381,10668.101612,0.23498,NaN
2,378.0,1060.576681,0.384186,43.616806,30.294941,66.567925,60.863155,57.135169,53.128827,50.114649,...,391.001621,703.641577,311.695918,404.38948,284.198075,-32841.046769,43509.148381,10668.101612,0.23498,NaN
3,504.0,1061.007118,0.382300,43.753723,30.419193,66.553718,60.930228,57.186279,53.162740,50.165256,...,391.001621,703.641577,311.695918,404.38948,284.198075,-32841.046769,43509.148381,10668.101612,0.23498,NaN
4,630.0,1061.422942,0.381337,43.872257,30.516474,66.530784,60.987319,57.224939,53.181550,50.199161,...,391.001621,703.641577,311.695918,404.38948,284.198075,-32841.046769,43509.148381,10668.101612,0.23498,NaN


##### **Definindo as Colunas que serão utilizadas de cada dataframe**

In [23]:
usecols_entry_data = [
"Desal - Hot feedwater outlet temperature (°C)",
"Desal - Cold feedwater inlet temperature (°C)",
"Desal - Cold feedwater outlet temperature (°C)",
"Desal - Hot feedwater outlet salinity (g/L)",
"Desal - Cold feedwater outlet salinity (g/L)",
"Desal - Distilled water production rate (kg/h)",
"Desal - Hot feedwater outlet mass flow rate (kg/h)",
"FeedTank - Water salinity (g/L)",
"FeedTank - Water temperature (°C)",
"ColdHex - Heat transfer rate (W)",
"ColdHex - Hot side inlet temperature (°C)",
"ColdHex - Cold side outlet temperature (°C)",
"HotHex - Heat transfer rate (W)",
"HotHex - Hot side inlet temperature (°C)",
"HotHex - Cold side outlet salinity (g/L)",
"HotHexBypass - Outlet temperature (°C)",
"CoolTower - Heat transfer rate (W)",
"CoolTower - Hot side inlet temperature (°C)",
"CoolTower - Cold side outlet temperature (°C)",
"CoolTowerBypass - mass flow rate (kg/h)",
"WHRU - Heat transfer rate (W)",
"WHRU - Hot side outlet temperature (°C)",
"WHRU - Cold side inlet temperature (°C)",
"WHRU - Cold side outlet temperature (°C)",
"Turbine - Regenerator hot side outlet temperature (°C)",
"Turbine - Net power (W)"
]

usecols_report = [
"Reposition water temperature at the entrance of the regulating tank =",
"Cooling heat exchanger flow rate =",
"Temperature at the entrance of the cold side =",
"Cool tower mass flow rate =",
"Mass flow rate at the cold side of the WHRU (line mass flow rate) =",
"WHRU mass flow rate =",
"heat load ="
]

##### Filtrando os Dataframes de Acordo com as Colunas